# DA5401 Assignment 6
## Imputation via Regression for Missing Data

In [340]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report

In [341]:
df = pd.read_csv('/content/UCI_Credit_Card.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

In [342]:
df.isnull().sum()

,0
ID,0
LIMIT_BAL,0
SEX,0
EDUCATION,0
MARRIAGE,0
AGE,0
PAY_0,0
PAY_2,0
PAY_3,0
PAY_4,0


In [343]:
cols_to_nan = ['AGE', 'BILL_AMT1', 'PAY_AMT4']

for col in cols_to_nan:
    df.loc[df.sample(frac=0.07).index,col] = np.nan

print(df[cols_to_nan].isnull().sum())

AGE          2100
BILL_AMT1    2100
PAY_AMT4     2100
dtype: int64


In [344]:
target_col = 'default.payment.next.month'
X = df.drop(columns=[target_col])
y = df[target_col]

In [345]:
df_median = df.copy()
for col in cols_to_nan:
    median_val = df_median[col].median()
    df_median[col] = df_median[col].fillna(median_val)

### 1) Simple Imputation: Why Median Is Preferred Over Mean

In this preprocessing step, missing values in numerical features were filled using the **median** instead of the **mean**.

####  Why use the median?
- The **median** is a robust central value that is **less influenced by outliers** or **skewed data**.
- In financial datasets (e.g., credit card default data), variables like **bill amounts** and **payment amounts** often have **long-tailed distributions** with extreme values.
- These extreme values can **distort the mean**, making it an unreliable measure for imputation.

####  Practical Advantage
> Mean imputation can distort the feature distribution by reducing variance and shifting the overall mean, which may introduce bias into the model.  
> Median imputation maintains the **original data distribution** better and is **more stable in the presence of extreme values**.

 Therefore, **median imputation** is a better choice for **skewed or heterogeneous datasets** commonly found in real-world applications.


In [346]:
df_linear = df.copy()
col_to_impute = 'AGE'

df_known = df_linear[df_linear[col_to_impute].notnull()]
df_missing = df_linear[df_linear[col_to_impute].isnull()]

X_known = df_known.drop(columns=[col_to_impute, target_col])
y_known = df_known[col_to_impute]

lin_reg = LinearRegression()
lin_reg.fit(X_known.select_dtypes(include=[np.number]).fillna(0), y_known)

X_missing = df_missing.drop(columns=[col_to_impute, target_col])
preds = lin_reg.predict(X_missing.select_dtypes(include=[np.number]).fillna(0))

df_linear.loc[df_linear[col_to_impute].isnull(), col_to_impute] = preds

###2) Linear Regression Imputation and the "Missing At Random (MAR)" Assumption

To handle missing values in the **AGE** column, a **Linear Regression model** was used to estimate and fill in the missing entries based on other available features.

####  Idea Behind the Method
A predictive model is trained using rows where AGE is already known. The model learns a relationship of the form:
$$
\hat{AGE} = g(\text{other features}) + \epsilon
$$

This learned relationship is then used to estimate missing AGE values.


####  MAR Assumption (Missing At Random)

This approach relies on the **Missing At Random (MAR)** assumption:

- The reason a value is missing depends only on **other observed variables** in the dataset, and **not on the missing value itself**.
- Example: If younger customers tend to skip entering AGE more often **because they have smaller credit limits (an observed feature)**, then AGE is missing at random with respect to credit limit → MAR holds.
- MAR allows us to **use other features to predict missing values** safely.


####  Why MAR Matters

-  If data are **MAR**, regression imputation can produce **reasonable and unbiased estimates**.
-  If data are **MNAR (Missing Not at Random)**—meaning the missingness depends on the value of AGE itself—then predictions may be biased.


####  Takeaway

> Linear Regression imputation uses relationships between features to produce smarter estimates of missing values compared to simple median/mean imputation.  
> This method is valid **only if the missing values satisfy the MAR assumption**, meaning the missingness can be explained by other observed variables.


In [347]:
df_knn = df.copy()
col_to_impute = 'AGE'

df_known = df_knn[df_knn[col_to_impute].notnull()]
df_missing = df_knn[df_knn[col_to_impute].isnull()]

X_known = df_known.drop(columns=[col_to_impute, target_col])
y_known = df_known[col_to_impute]

knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_known.select_dtypes(include=[np.number]).fillna(0), y_known)

X_missing = df_missing.drop(columns=[col_to_impute, target_col])
preds_knn = knn_reg.predict(X_missing.select_dtypes(include=[np.number]).fillna(0))

df_knn.loc[df_knn[col_to_impute].isnull(), col_to_impute] = preds_knn

In [349]:
df_listwise = df.dropna()

In [353]:
datasets = {
    "A_Median": df_median,
    "B_Linear": df_linear,
    "C_KNN": df_knn,
    "D_Listwise": df_listwise
}

results = {}
target_col = 'default.payment.next.month'

# Common split indices based on Dataset A
X_full = df_median.drop(columns=[target_col])
y_full = df_median[target_col]
train_idx, test_idx = train_test_split(
    np.arange(len(X_full)), test_size=0.2, random_state=42, stratify=y_full
)

for name, data in datasets.items():
    df_temp = data.copy()

    # 🔹 For regression-imputed datasets, drop rows with remaining NaNs
    if name in ["B_Linear", "C_KNN"]:
        df_temp = df_temp.dropna()

    # Reset index to align indices cleanly
    df_temp = df_temp.reset_index(drop=True)

    # Align to same indices (clip if dataset is smaller after dropping)
    common_idx = [i for i in train_idx if i < len(df_temp)]
    common_test_idx = [i for i in test_idx if i < len(df_temp)]

    X_train = df_temp.drop(columns=[target_col]).iloc[common_idx]
    X_test = df_temp.drop(columns=[target_col]).iloc[common_test_idx]
    y_train = df_temp[target_col].iloc[common_idx]
    y_test = df_temp[target_col].iloc[common_test_idx]

    # Standardize numeric columns
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.select_dtypes(include=[np.number]))
    X_test = scaler.transform(X_test.select_dtypes(include=[np.number]))

    # Logistic Regression model
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Classification report
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = report

    print(f"\n=== {name} ===")
    print(classification_report(y_test, y_pred, digits=4))



=== A_Median ===
              precision    recall  f1-score   support

           0     0.8182    0.9698    0.8876      4673
           1     0.6941    0.2411    0.3579      1327

    accuracy                         0.8087      6000
   macro avg     0.7562    0.6055    0.6228      6000
weighted avg     0.7908    0.8087    0.7704      6000


=== B_Linear ===
              precision    recall  f1-score   support

           0     0.8289    0.9720    0.8948      4042
           1     0.7237    0.2674    0.3905      1107

    accuracy                         0.8205      5149
   macro avg     0.7763    0.6197    0.6426      5149
weighted avg     0.8063    0.8205    0.7864      5149


=== C_KNN ===
              precision    recall  f1-score   support

           0     0.8285    0.9718    0.8945      4042
           1     0.7206    0.2656    0.3881      1107

    accuracy                         0.8200      5149
   macro avg     0.7746    0.6187    0.6413      5149
weighted avg     0.8053

In [354]:
# Summarize key performance metrics for all models
performance_summary = []

for dataset_name, report in results.items():
    performance_summary.append({
        "Dataset": dataset_name,
        "Accuracy": round(report["accuracy"], 4),
        "Precision (Class 1)": round(report["1"]["precision"], 4),
        "Recall (Class 1)": round(report["1"]["recall"], 4),
        "F1-Score (Class 1)": round(report["1"]["f1-score"], 4)
    })

summary_table = pd.DataFrame(performance_summary)
summary_table = summary_table.set_index("Dataset")

print("\n=== Comparative Performance Across Imputation Methods ===")
display(summary_table)



=== Comparative Performance Across Imputation Methods ===


,Accuracy,Precision (Class 1),Recall (Class 1),F1-Score (Class 1)
Dataset,,,,
A_Median,0.8087,0.6941,0.2411,0.3579
B_Linear,0.8205,0.7237,0.2674,0.3905
C_KNN,0.8200,0.7206,0.2656,0.3881
D_Listwise,0.8023,0.7030,0.2358,0.3532


##  Part C-2 : Evaluation of Imputation Strategies

### 1) Balancing Listwise Deletion and Imputation
From the comparative table, the **Listwise Deletion approach (D_Listwise)** achieves the raw accuracy of (0.8023), but its **F1-score (0.3532)** is lower than all imputed datasets (A-C).  
This reflects a common trade-off:

* **Listwise Deletion** removes any row with missing entries.  
  * **Pros:** Straightforward, no assumptions about how data is missing.  
  * **Cons:** Reduces dataset size and statistical power. Important samples are discarded, which can skew class proportions toward the majority (non-default) class.  
  * Consequently, while overall accuracy may appear decent, recall for the minority “default = 1” class is diminished, leading to a lower F1-score.

* **Imputation approaches (A-C)** retain the full dataset and preserve underlying distributions.  
  * By estimating plausible replacements for missing values, feature–target relationships are better maintained.  
  * As a result, while accuracy remains similar, F1-scores are higher, indicating improved detection of defaulters.

 **Insight:** Even if Listwise Deletion shows slightly decent accuracy, its reduced recall and F1 make it less suitable for tasks where identifying minority-class events (defaults) is critical.


###2) Linear vs Non-Linear Imputation
Comparison of **Linear Regression (B)** and **KNN Regression (C)**:

| Model | F1-Score (Class 1) |
|:------|:------------------|
| Linear Regression | **0.3905** |
| KNN Regression | 0.3881 |

* The difference is minor, but **Linear Regression edges out KNN slightly**.  
* This indicates the relationship between the imputed feature (**AGE**) and other predictors is roughly linear.  
* KNN, being non-linear, may be more sensitive to local noise and feature scaling, leading to slightly lower F1 performance.

**Conclusion:** Linear imputation is more reliable here due to the approximately linear associations between variables.


### 3) Recommended Approach for Missing Values
Taking both results and theory into account:

* **Median Imputation (A)** is a strong baseline—robust against outliers and straightforward to implement.  
* **Linear Regression Imputation (B)** achieves the **highest F1-score**, demonstrating that leveraging inter-feature correlations improves predictive quality.  
* **KNN Imputation (C)** performs almost equivalently, suggesting non-linear patterns are minimal for the missing feature.  
* **Listwise Deletion (D)**, though easy to apply, discards valuable information and can bias results.

 **Recommendation:** Use **regression-based imputation**, preferably **Linear Regression**, for features with missing values that are roughly linearly related to other predictors.  
This approach maintains dataset size, preserves relationships among variables, and optimizes detection of minority-class events.



### 4) Performance Overview

| Model | Accuracy | Precision (Class 1) | Recall (Class 1) | F1-Score (Class 1) |
|:------|:--------|:------------------|:----------------|:-----------------|
| A (Median Imputation) | 0.8087 | 0.6941 | 0.2411 | 0.3579 |
| B (Linear Regression) | 0.8205 | 0.7237 | 0.2674 | **0.3905** |
| C (KNN Regression) | 0.8200 | 0.7206 | 0.2656 | 0.3881 |
| D (Listwise Deletion) | 0.8023 | 0.7030 | 0.2358 | 0.3532 |

*Values rounded to 4 decimals.*
